# InfoGAN Training Notebook
The ```train.py``` script changed into a jupyter notebook to make interactively tweaking the training and adjusting the data a bit easier.

In [1]:
import os, sys
import numpy as np
import tensorflow as tf
from collections import namedtuple
sys.path.append('..') # add the parent directory to the path
def create_gan_settings(**kwargs):
    arg_list = kwargs.items()
    arg_type = namedtuple('InfoGANSettings', [k for (k, _) in arg_list])
    return arg_type(*[v for (_, v) in arg_list])

args = create_gan_settings(epochs = 100, dataset = None,
                           max_images = None, scale_dataset = [28, 28],
                           batch_size = 64, generator_lr = 1e-3,
                           discriminator_lr = 2e-4, categorical_lambda = 1.0,
                           continuous_lambda = 1.0, categorical_cardinality = [10],
                           generator = "fc:1024,fc:7x7x128,reshape:7:7:128,deconv:4:2:64,deconv:4:2:1:sigmoid",
                           discriminator = "conv:4:2:64:lrelu,conv:4:2:128:lrelu,fc:1024:lrelu",
                           num_continuous = 2, seed = 1234, style_size = 62, plot_every = 200,
                           infogan = True, use_batch_norm = True, fix_std = True, force_grayscale = False)


args

InfoGANSettings(discriminator_lr=0.0002, style_size=62, force_grayscale=False, batch_size=64, generator='fc:1024,fc:7x7x128,reshape:7:7:128,deconv:4:2:64,deconv:4:2:1:sigmoid', categorical_cardinality=[10], dataset=None, infogan=True, categorical_lambda=1.0, continuous_lambda=1.0, fix_std=True, epochs=100, seed=1234, num_continuous=2, generator_lr=0.001, plot_every=200, discriminator='conv:4:2:64:lrelu,conv:4:2:128:lrelu,fc:1024:lrelu', scale_dataset=[28, 28], max_images=None, use_batch_norm=True)

In [2]:
import infogan as ifg
from infogan.tf_utils import scope_variables
from infogan.categorical_grid_plots import CategoricalPlotter
from infogan.tf_utils import (
    scope_variables,
    NOOP,
    load_mnist_dataset,
    run_network,
    leaky_rectify,
)
from infogan.misc_utils import (
    next_unused_name,
    add_boolean_cli_arg,
    create_progress_bar,
    load_image_dataset,
)
from infogan.noise_utils import (
    create_infogan_noise_sample,
    create_gan_noise_sample,
)

# Setup
## Loading training data 

In [3]:
np.random.seed(args.seed)
batch_size = args.batch_size
n_epochs = args.epochs
use_batch_norm = args.use_batch_norm
fix_std = args.fix_std
plot_every = args.plot_every
use_infogan = args.infogan
style_size = args.style_size
categorical_cardinality = args.categorical_cardinality
num_continuous = args.num_continuous
generator_desc = args.generator
discriminator_desc = args.discriminator
if args.dataset is None:
    assert args.scale_dataset == [28, 28]
    X = ifg.load_mnist_dataset()
    if args.max_images is not None:
        X = X[:args.max_images]
    dataset_name = "mnist"
else:
    scaled_image_width, scaled_image_height = args.scale_dataset

    # load pngs and jpegs here
    X = load_image_dataset(
        args.dataset,
        desired_width=scaled_image_width, # TODO(jonathan): pick up from generator or add a command line arg (either or)...
        desired_height=scaled_image_height,
        value_range=(0.0, 1.0),
        max_images=args.max_images,
        force_grayscale=args.force_grayscale
    )
    dataset_name = basename(args.dataset.rstrip("/"))

if use_infogan:
    z_size = style_size + sum(categorical_cardinality) + num_continuous
    sample_noise = ifg.create_infogan_noise_sample(
        categorical_cardinality,
        num_continuous,
        style_size
    )
else:
    z_size = style_size
    sample_noise = create_gan_noise_sample(style_size)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Building Model
The model is built from the arguments and the training data

In [4]:
discriminator_lr = tf.get_variable(
    "discriminator_lr", (),
    initializer=tf.constant_initializer(args.discriminator_lr)
)
generator_lr = tf.get_variable(
    "generator_lr", (),
    initializer=tf.constant_initializer(args.generator_lr)
)

n_images, image_height, image_width, n_channels = X.shape

discriminator_lr_placeholder = tf.placeholder(tf.float32, (), name="discriminator_lr")
generator_lr_placeholder = tf.placeholder(tf.float32, (), name="generator_lr")
assign_discriminator_lr_op = discriminator_lr.assign(discriminator_lr_placeholder)
assign_generator_lr_op = generator_lr.assign(generator_lr_placeholder)

## begin model
true_images = tf.placeholder(
    tf.float32,
    [None, image_height, image_width, n_channels],
    name="true_images"
)
zc_vectors = tf.placeholder(
    tf.float32,
    [None, z_size],
    name="zc_vectors"
)
is_training_discriminator = tf.placeholder(
    tf.bool,
    [],
    name="is_training_discriminator"
)
is_training_generator = tf.placeholder(
    tf.bool,
    [],
    name="is_training_generator"
)

fake_images = ifg.generator_forward(
    zc_vectors,
    generator_desc,
    is_training=is_training_generator,
    name="generator",
    debug=True
)

print("Generator produced images of shape %s" % (fake_images.get_shape()[1:]))
print("")

discriminator_fake = ifg.discriminator_forward(
    fake_images,
    discriminator_desc,
    is_training=is_training_discriminator,
    name="discriminator",
    use_batch_norm=use_batch_norm,
    debug=True
)
prob_fake = discriminator_fake["prob"]
discriminator_true = ifg.discriminator_forward(
    true_images,
    discriminator_desc,
    is_training=is_training_discriminator,
    reuse=True,
    name="discriminator",
    use_batch_norm=use_batch_norm
)
prob_true = discriminator_true["prob"]

generator architecture
Fully connected with num_outputs=1024 followed by relu
Fully connected with num_outputs=6272 followed by relu
Reshape to [7, 7, 128]
Deconvolution with nkernels=4, stride=2, num_outputs=64 followed by relu
Deconvolution with nkernels=4, stride=2, num_outputs=1 followed by sigmoid

Generator produced images of shape (28, 28, 1)

discriminator architecture
Convolution with nkernels=4, stride=2, num_outputs=64 followed by lrelu
Convolution with nkernels=4, stride=2, num_outputs=128 followed by lrelu
Fully connected with num_outputs=1024 followed by lrelu



## Objective Functions
For the generator and discriminator the appropriate objective and loss functions need to be established

In [5]:
# discriminator should maximize:
ll_believing_fake_images_are_fake = tf.log(1.0 - prob_fake + ifg.TINY)
ll_true_images = tf.log(prob_true + ifg.TINY)
discriminator_obj = (
    tf.reduce_mean(ll_believing_fake_images_are_fake) +
    tf.reduce_mean(ll_true_images)
)

# generator should maximize:
ll_believing_fake_images_are_real = tf.reduce_mean(tf.log(prob_fake + ifg.TINY))
generator_obj = ll_believing_fake_images_are_real

discriminator_solver = tf.train.AdamOptimizer(
    learning_rate=discriminator_lr,
    beta1=0.5
)
generator_solver = tf.train.AdamOptimizer(
    learning_rate=generator_lr,
    beta1=0.5
)

discriminator_variables = scope_variables("discriminator")
generator_variables = scope_variables("generator")

train_discriminator = discriminator_solver.minimize(-discriminator_obj, var_list=discriminator_variables)
train_generator = generator_solver.minimize(-generator_obj, var_list=generator_variables)
discriminator_obj_summary = tf.summary.scalar("discriminator_objective", discriminator_obj)
generator_obj_summary = tf.summary.scalar("generator_objective", generator_obj)

## Tensorboard Outputs
The outputs for tensorboard are configured so the results can be visualized there

In [6]:
if use_infogan:
    categorical_c_vectors = []
    offset = 0
    for cardinality in categorical_cardinality:
        categorical_c_vectors.append(
            zc_vectors[:, offset:offset+cardinality]
        )
        offset += cardinality

    continuous_c_vector = zc_vectors[:, offset:offset + num_continuous]

    q_output = ifg.reconstruct_mutual_info(
        categorical_c_vectors,
        continuous_c_vector,
        categorical_lambda=args.categorical_lambda,
        continuous_lambda=args.continuous_lambda,
        fix_std=fix_std,
        hidden=discriminator_fake["hidden"],
        is_training=is_training_discriminator,
        name="mutual_info"
    )

    mutual_info_objective = q_output["mutual_info"]
    mutual_info_variables = scope_variables("mutual_info")
    neg_mutual_info_objective = -mutual_info_objective
    train_mutual_info = generator_solver.minimize(
        neg_mutual_info_objective,
        var_list=generator_variables + discriminator_variables + mutual_info_variables
    )
    ll_categorical = q_output["ll_categorical"]
    ll_continuous = q_output["ll_continuous"]
    std_contig = q_output["std_contig"]

    mutual_info_obj_summary = tf.summary.scalar("mutual_info_objective", mutual_info_objective)
    ll_categorical_obj_summary = tf.summary.scalar("ll_categorical_objective", ll_categorical)
    ll_continuous_obj_summary = tf.summary.scalar("ll_continuous_objective", ll_continuous)
    std_contig_summary = tf.summary.scalar("std_contig", std_contig)
    generator_obj_summary = tf.summary.merge([
        generator_obj_summary,
        mutual_info_obj_summary,
        ll_categorical_obj_summary,
        ll_continuous_obj_summary,
        std_contig_summary
    ])
else:
    neg_mutual_info_objective = NOOP
    mutual_info_objective = NOOP
    train_mutual_info = NOOP
    ll_categorical = NOOP
    ll_continuous = NOOP
    std_contig = NOOP
    entropy = NOOP

In [7]:
log_dir = next_unused_name(
    os.path.join(
        ifg.PROJECT_DIR,
        "%s_log" % (dataset_name,),
        "infogan" if use_infogan else "gan"
    )
)
journalist = tf.summary.FileWriter(
    log_dir,
    flush_secs=10
)
print("Saving tensorboard logs to %r" % (log_dir,))

Saving tensorboard logs to 'D:\\CS7476\\proj_exp\\tensorflow-infogan\\mnist_log\\infogan-1'


In [8]:
img_summaries = {}
if use_infogan:
    plotter = CategoricalPlotter(
        categorical_cardinality=categorical_cardinality,
        num_continuous=num_continuous,
        style_size=style_size,
        journalist=journalist,
        generate=lambda sess, x: sess.run(
            fake_images,
            {zc_vectors:x, is_training_discriminator:False, is_training_generator:False}
        )
    )
else:
    image_placeholder = None
    plotter = None
    img_summaries["fake_images"] = tf.summary.image("fake images", fake_images, max_images=10)
image_summary_op = tf.summary.merge(list(img_summaries.values())) if len(img_summaries) else NOOP


In [10]:
idxes = np.arange(n_images, dtype=np.int32)
iters = 0
with tf.Session() as sess:
    # pleasure
    sess.run(tf.global_variables_initializer())
    # content
    for epoch in range(n_epochs):
        disc_epoch_obj = []
        gen_epoch_obj = []
        infogan_epoch_obj = []

        np.random.shuffle(idxes)
        pbar = create_progress_bar("epoch %d >> " % (epoch,))

        for idx in pbar(range(0, n_images, batch_size)):
            batch = X[idxes[idx:idx + batch_size]]
            # train discriminator
            noise = sample_noise(batch_size)
            _, summary_result1, disc_obj, infogan_obj = sess.run(
                [train_discriminator, discriminator_obj_summary, discriminator_obj, neg_mutual_info_objective],
                feed_dict={
                    true_images:batch,
                    zc_vectors:noise,
                    is_training_discriminator:True,
                    is_training_generator:True
                }
            )

            disc_epoch_obj.append(disc_obj)

            if use_infogan:
                infogan_epoch_obj.append(infogan_obj)

            # train generator
            noise = sample_noise(batch_size)
            _, _, summary_result2, gen_obj, infogan_obj = sess.run(
                [train_generator, train_mutual_info, generator_obj_summary, generator_obj, neg_mutual_info_objective],
                feed_dict={
                    zc_vectors:noise,
                    is_training_discriminator:True,
                    is_training_generator:True
                }
            )

            journalist.add_summary(summary_result1, iters)
            journalist.add_summary(summary_result2, iters)
            journalist.flush()
            gen_epoch_obj.append(gen_obj)

            if use_infogan:
                infogan_epoch_obj.append(infogan_obj)

            iters += 1

            if iters % plot_every == 0:
                if use_infogan:
                    plotter.generate_images(sess, 10, iteration=iters)
                else:
                    noise = sample_noise(batch_size)
                    current_summary = sess.run(
                        image_summary_op,
                        {
                            zc_vectors:noise,
                            is_training_discriminator:False,
                            is_training_generator:False
                        }
                    )
                    journalist.add_summary(current_summary, iters)
                journalist.flush()

        msg = "epoch %d >> discriminator LL %.2f (lr=%.6f), generator LL %.2f (lr=%.6f)" % (
            epoch,
            np.mean(disc_epoch_obj), sess.run(discriminator_lr),
            np.mean(gen_epoch_obj), sess.run(generator_lr)
        )
        if use_infogan:
            msg = msg + ", infogan loss %.2f" % (np.mean(infogan_epoch_obj),)
        print(msg)


KeyboardInterrupt: 

In [ ]:
# Show the tensorboard results
!tensorboard --logdir mnist_log

In [6]:
# Reset everything to initial state
tf.reset_default_graph()

In [4]:
X.shape

(55000, 28, 28, 1)

In [5]:
X.min()

0.0

In [6]:
X.max()

1.0